# Introduction

This is a personal project to grasp some concept of SET50 Options pricing.  The historical Options price was mannually collected by myself on a daily basis with a Google Sheet during those trading days. 

Here is how it lool like on a speadsheet:

![options_sheet](options_sheet.png)

I have been trading Options for several years and still wanting to better understand the pricing of the Options, so, as a data analyst, I think I will use some of the DA tools to help me grasp a better picture of Options Pricing.

----------------

Options is a derivative origanally aim to be used as an insurance for an underlying asset volatility during a certain period of time. And just like any insurances, the buyer need to pay the insurance premium to get insured, but, a million dollar question is, "How much" you should pay or sell for an option at a certain time.

Since Options premium depends a lot on: 
- underlying's volatility and 
- time to expiration 
This notebook thus aim to map Options premium to both of the factors.

For anyone who want to learn more about Options, here's a [link](https://www.investopedia.com/terms/s/stockoption.asp) to lay some foundation. 

----------------

**Thailand Future Exchange(TFEX)** is a derivative market wich provides wide range of derivative such as "Index Future" and "Index Options". Each contracts expired at the end of each quarters and coded by the letter and numbers. Here is how to read the contract symbol:

 * S50H20 -> Means: this is an SET50 Index Future Expired at March 2020.
 
 * S50M20C900 -> Means: this is an SET50 Index Options at Strike Price 900 Expired at June 2020.
 
 The month code are as follows:
 
 * H -> March Expiration
 * M -> June Expiration
 * U -> September Expiration
 * Z -> December Expiration
 
 Thus **S50U22** is an **SET50 Index Future Contract expired at September 2022.**
 
 
 For more detailed explanation, please visit the [exchange](https://www.tfex.co.th/tfex/index.html?locale=en_US) tutorial.

-----------------------

Thanak Rattanopastkul, 2022 

# Dataset's Note

There are 2 datasets in this notebook.
* Merged-Table 1.csv
* option_premium.csv

### Merged-Table 1.csv
This dataset is a historical price of SET50 Futures, an equity index future of Thailand's Stock Exchange. I Collected directly from the **TFEX**(Thailand's Future Exchange) [website](https://www.tfex.co.th/tfex/dailyMarketReport.html?locale=en_US) and merge all the series together and add a trading SYMBOL to every observations.
    
The collumns in Merged-Table 1.csv are as follows:
* **Date** -> Trading day
* **Open** -> Opening price
* **High** -> Higest price duing the day
* **Low** -> Lowest price duing the day
* **Close** -> Closing price
* **SP** -> [Settlement price](https://www.investopedia.com/terms/s/settlementprice.asp)
* **Chg** -> Price change from last trading day
* **%Chg** -> Price percent change from last trading day
* **Vol** -> Trading volume
* **OI** -> [Open Interest](https://www.investopedia.com/terms/o/openinterest.asp)
* **Series** -> Trading symbol

### option_premiun.csv
This is a dataset manually collected by myself at the end of each trading day to record the price movement and any market beheaviors. I take only the relevant features and exported the dataset to CSV file.

The collumns in Merged-Table 1.csv are as follows:
* **Date** -> Trading day
* **Series Name** -> Trading Symbol
* **Underlying Close** -> Nearest SET50 Future Contract closing price 
* **Expiration** -> Options Expiration Date
* **Day to expire** -> Day count to expiration
* **Underlying Change** -> Nearest SET50 Future Contract price change from last observation
* **ATM premium** -> SUM of Put and Call Options Premium at the [ATM](https://www.investopedia.com/terms/a/atthemoney.asp)
* **+/-1 OTM premium** -> SUM of Put and Call Options Premium at the [+1 Call and -1 Put](https://www.investopedia.com/terms/o/outofthemoney.asp)


### Import Dependencies

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

import os
import gc

import warnings
warnings.filterwarnings('ignore')

### Read the Data

In [31]:
path_1 = os.getcwd()+'/Merged-Table 1.csv'
merged_future = pd.read_csv(path_1, 
                  parse_dates=['Date'],
                  infer_datetime_format=True,
                  dayfirst=True,
)

path_2 = os.getcwd()+'/option_premium.csv'

manual_data = pd.read_csv(path_2, 
                  parse_dates=['date','Expiration'],
                  infer_datetime_format=True,
                  dayfirst=True
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/noonpritsana/Desktop/Nak/options_project/options_analysis/Merged-Table 1.csv'

In [ ]:
merged_future.head()

In [ ]:
merged_future.info()

In [ ]:
manual_data.head()

In [ ]:
manual_data.info()

Many fof the merge_future data are in from of stings. Need to change it in to float and int.

Eyeballing the CSV, there are some value in **Chg** and **%Chg** are "-" let's see if they can be recalculated. 

In [ ]:
dashed_index =  merged_future.loc[(merged_future['Chg'] == '-') | (merged_future['%Chg']=='-')].index

for i in dashed_index:
    display(merged_future.loc[i-1:i+1])

Seems like the recalculation is possible. Let's change the dtype to string and calculate the Chg and %Chg.

In [ ]:
for i in merged_future.index:
    
        if merged_future.loc[i, 'Chg'] != '-':
            for col in ['Open', 'High', 'Low', 'Close', 'SP', 'Chg','%Chg','Vol','OI']:
                merged_future.loc[i, col] = float(merged_future.loc[i, col].replace('"','').replace(',',''))
        else:
            col2 = ['Open', 'High', 'Low', 'Close', 'SP','Vol','OI']
            for col2 in col2:   
                merged_future.loc[i, col2] = float(merged_future.loc[i, col2].replace('"','').replace(',',''))

            merged_future.loc[i, 'Chg'] = float(9.9) #temp
            merged_future.loc[i, '%Chg'] = float(9.9) #temp
            merged_future.loc[i, 'Chg'] = merged_future.loc[i, 'SP'] - merged_future.loc[i-1, 'SP']
            merged_future.loc[i, '%Chg'] = (merged_future.loc[i, 'SP'] - merged_future.loc[i-1, 'SP'])/merged_future.loc[i-1, 'SP']


Check if all the data are of dtype strings and no "-" on **Chg** and **%Chg**.

In [ ]:
merged_future.loc[dashed_index]

"merged_future" contains 722 observations from SET50 Index Future from 2019 to 2022.
"manual_data" contains 595 observations from SET0 Index Future and Options from 2019 to 2021.

Since "merged_future" contains better Future Contracts trading data, The plan is to validate the data and merge two tables into one using Future data from "merged_future" and Options data from "manual_data".

### Check and Validate data

In [ ]:
merged_future.isna().sum()

In [ ]:
manual_data.isna().sum()

Only some data from the manually collected data are missing. For the missing "Underlying Close" and "Underlying Change" , we can replace with the "Close" and "Chg" from the other dataset.

Let's see the missing "ATM premium".

In [ ]:
manual_data.loc[manual_data['ATM premium'].isna()]

For the entries at 

* 2020-03-09 
* 2020-03-12
* 2020-03-23 

those are very special case when COVID-19 crash the global market. The market actually crash and [Circuit Breaker](https://www.investopedia.com/terms/c/circuitbreaker.asp) was implimented on those three days. The Options market maker stop providing liquidity. Thus, those three very rare occasions should be dropped.

In [ ]:
manual_data.dropna(inplace=True)

### Check Entry Duplications

In [ ]:
manual_data.loc[manual_data.duplicated()]

In [ ]:
merged_future.loc[merged_future.duplicated()]

### Merge Data

Will use date and Series as merging keys and extract only the needed columns.

In [ ]:
merged_future_options = pd.merge(left=manual_data,right=merged_future,left_on=['date','Series Name'],right_on=['Date','Series'])
merged_future_options.head(3)

In [ ]:
a = list(merged_future.columns) #all columns taken 
a.extend(['Expiration','Day to expire','ATM premium','+/-1 OTM premium']) #extend with only needed column from another dataset.
ready_df = merged_future_options[a]

# delete the unused variable
del [path_1,path_2,merged_future,manual_data,merged_future_options,dashed_index,col,col2]
gc.collect()

In [ ]:
ready_df.head()

### EDA

Let's put all the most important features together to see the pattern. 
* Underlying Price (Close)
* Trading Volume
* Open Interest
* Options price for both ATM an OTM


In [ ]:
interval = [i*25 for i in range(26,48)] #to vertically frame between 650 - 1248 and use as yticks

fig = plt.subplots(figsize=(15,15))
plt.subplot(311)
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, 'Close'],label=symbol)
plt.title('SET50 Future Close Price')
plt.grid()
plt.yticks(interval)
plt.vlines(['2020-03-09','2020-03-12','2020-03-23'],ymin=600,ymax=1200,label='Circuit Break',linestyles='dashed',colors='orange')
plt.legend()

plt.subplot(312)
plt.title('SET50 Future Trading Volume and Open Interest')
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, 'OI'],label=symbol+' OI')
plt.plot(ready_df.Date,ready_df.Vol,label='Volume')
plt.vlines(['2020-03-09','2020-03-12','2020-03-23'],ymin=20000,ymax=680000,label='Circuit Break',linestyles='dashed',colors='orange')
plt.grid()


plt.subplot(313)
plt.title('SET50 Options Price ATM total and OTM total')
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, 'ATM premium'],c='r',label='ATM premiun')
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, '+/-1 OTM premium'],c='0.5',label='OTM premium')
plt.vlines(['2020-03-09','2020-03-12','2020-03-23'],ymin=0,ymax=200,label='Circuit Break',linestyles='dashed',colors='orange')
plt.grid()

plt.show()

Let's see the price distribution.

In [ ]:
interval = [i*25 for i in range(24,46)]

fig = plt.subplots(figsize=(7,4))
plt.grid(axis='y')
plt.title('SET50 Future Close Distribution')
plt.hist(ready_df['Close'],bins=interval,orientation='vertical',linewidth=0.5,edgecolor='white')
plt.xticks(interval,rotation=90)

plt.show()

During Q3 2019 to Q3 2021, Price clustered around 900-975 and 1050-1100. However the left skew is due to the COVID market Crash. At the Crash, the lowest Close was around 675.

## What do we see form the EDA

The data are from mid Q3 2019 to Q3 2021. Represented by 9 series of contracts duing those 9 quarters. The SET50 price stay around 900-975 the most. This could be a concept to build a range bound trading strategy on.

**Other Remarkable Characteristics**

**1. The Covid Plunge**

There are a lot to take in from the above chart. First of all, the most eye-catching area is around March and April 2020 where there was a very steep drop in SET50 Future price from 1075 to 675 (around 37% loss) and the three vertical lines indicate the circuit breaker. That rare and unexpected event could be called a "Black Swan". We do not see that every day. However, after the plunge, the index seems to keep climbing up.

**2. Volume Spike**

It is expected that, the trading volume would peak during a panic sales. It did, from around 150k before the plunge, it peaked at over 600k at the Circuit Break.

**3. Future Contract Rollover**

At the expiration each quarter, all the future contract will be expired by settelement. This cause the OI of each future contract to go to zero and the next series gaining more OI.

**4. Options Premium**

The red lines are showing ATM while the grey lines are showing +/-1 OTM. All options price goes to zero at the expiration each quarter. However, whenever the SET50 Future Price rally, the Options price goes up too.

Option price is the reason for this analysis. Eventhough the most influential model to price derivatives are [Black-Schole-Merton](https://www.investopedia.com/terms/b/blackscholes.asp) I still find it too complicated to use in the real-time Options trading on my daily basis.

Therefore, this notebook is an attempt to create some personal and practical guideline for SET50 Options pricing.

### Feature Engineering

Options price affected the most by time to expiration and underlying volatility. Let's calculate the change in Options price and underlying "ATR" (Average True Range) to represent volatility.

Here is  the ATR formula:
$$
TR = max[(H-L) , abs(H-Cp) , abs(L-Cp)]
$$
$$
ATR = (\frac {1}{n}) \sum_{i=1}^{n}TR_{i}
$$

ATR is basically a rolling average of price range from last n days. The more volatile market, the wider range between the Highest and Lowest price, and reflect to the bigger ATR number. The universal default of the n in ATR is 14.
****
Further explanation of [ATR](https://www.investopedia.com/terms/a/atr.asp)

### Define ATR function

In [ ]:
def cal_tr(h,l,c):
    tr = max([h-l,abs(h-c),abs(l-c)])
    return tr

def df_atr(df,h_col,l_col,c_col,n=14):
    df['atr'] = None
    for i in df.index:
        df.loc[i, 'tr'] = cal_tr(h=df.loc[i, h_col],
                                    l=df.loc[i, l_col],
                                    c=df.loc[i, c_col])
        df['atr'] = df['tr'].rolling(n).mean()

In [ ]:
df_atr(ready_df,'High','Low','Close',14)
ready_df.tail()

ATR is done. 

Since ATM Options premium has two part. Intrinsic Value and Time Value. Another crucial information is Time Value for ATM options since this is the main profit for Options traders. Time value in a word is an Options premium that exceed the diffecence between the Strikeprice and the Underlying price. For more infomation about Time Value please follow this [link](https://www.investopedia.com/terms/t/timevalue.asp)

### Calculate ATM Time Value

Since the Strikeprice are thos multiples of 25, Therefore, at any underlying price : 
$$
TV = ATM.Premium - (Nearest.Strike - Close)
$$

For example at index 523 above: 
* Close = 968.6
* Nearest Strike = 975
* ATM = 8.5

$$
TV = 8.5-(975-968.6) = 2.1
$$

Noted that this is at 1 day before expiration, so the premium is very low.

In [ ]:
for i in ready_df.index:
    ready_df.loc[i, 'atm_tv'] = ready_df.loc[i, 'ATM premium']-abs((25*round(ready_df.loc[i, 'Close']/25))-ready_df.loc[i, 'Close']) #25*round(ready_df.loc[i, 'Close']/25) is to round to the nearest strike

ready_df.head()

Out of curiosity, let's see ATM OTM and TV together.

In [ ]:
plt.figure(figsize=(20,7))
plt.title('SET50 Options Price ATM total, OTM total and Time Value')
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, 'ATM premium'],c='r',label='ATM premiun')
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, 'atm_tv'],c='g',label='TV')
for symbol in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series'] == symbol, 'Date'],
                    ready_df.loc[ready_df['Series'] == symbol, '+/-1 OTM premium'],c='0.5',label='OTM premium')
plt.vlines(['2020-03-09','2020-03-12','2020-03-23'],ymin=0,ymax=200,label='Circuit Break',linestyles='dashed',colors='orange')
plt.grid()

plt.show()

TV is a lot more volatile compared to ATM. From now on, this notebook will be focusing only on Time Value as an Option Price. The +/-1 OTM Premium is already time value without any intrinsic value.

Next let's cast the "Days to expire" column from str to int for later use in analytics.



In [ ]:
for i in ready_df.index:    
    ready_df.loc[i, 'dte'] = int(ready_df.loc[i, 'Day to expire'].replace("'",'').replace('d','').replace('ms',''))

Next, let's create another column day from the quarter start (dfqs). This information will be use to compare across series for Options pice at the same interval comparaed to the start of the quarter. 

In [ ]:
# reference: a snippet from Alaxander at https://stackoverflow.com/questions/46167550/day-number-of-a-quarter-for-a-given-date-in-pandas

ready_df['q_day'] = [int((date - quarter_period.start_time).days + 1) for date, quarter_period in zip(ready_df['Date'], pd.PeriodIndex(ready_df['Date'], freq='Q'))]

# note for later code reading
# quarter_period is an insntance of a querter at a given time
# quarter_period.start_time returns the first day of that period.
# Thus ready_df['Date']-quarter_perid.startime+1 returns a date count of a given day from it's quarter beginning. +1 is to offset the first_day - first_day = 0



### EDA Continued

Since we have some new features from the engineering, let's explore them beginning with the relationship between TV and ATR at each day to expiration.

In [ ]:
# drop first 13 lines with missing ATR 
ready_df.dropna(inplace=True)
ready_df

In [ ]:
fig = plt.figure(figsize=(18,7))

ax1 = plt.subplot(121)
for sym in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series']==sym, 'q_day'], ready_df.loc[ready_df['Series']==sym , 'atm_tv'].rolling(5).mean(), label=sym,linewidth=1)
plt.title('ATM TV at different day from quarter begins')
plt.xticks(np.linspace(0,90,10))
plt.xlabel('Quarter Day')
plt.ylabel('Time Value at ATM')
plt.vlines(x=[30,60,90],ymin=0,ymax=150,linestyles='dashed')

ax2 = plt.subplot(122)
for sym in ready_df['Series'].unique():
    plt.plot(ready_df.loc[ready_df['Series']==sym, 'q_day'], ready_df.loc[ready_df['Series']==sym , '+/-1 OTM premium'].rolling(5).mean(), label=sym,linewidth=1)
plt.title('+/-1 OTM Premium at different day from quarter begins')
plt.xticks(np.linspace(0,90,10))
plt.xlabel('Quarter Day')
plt.ylabel('Time Value at +/-1 OTM')
plt.vlines(x=[30,60,90],ymin=0,ymax=150,linestyles='dashed')
ax2.legend()

plt.tight_layout(pad=2)
plt.show()    


Most of the premiums declines to zero over the period of quarter. On special occastion of COVID-19 at the last month of H20 (March 2020) the premium jumped fom around 20 to almost 80 at as the contract is just around 10 days to the expiration.

This effect continue on to M20 and U20. We saw ATM premium around 140 and 90 respectively at the beginning of the quarter instead of aroud 45.


The chart axes show only premium and day to expiration. Let's try 3D plot a bit.

### ciurrent working line

In [ ]:
%matplotlib widget 
# use this widget for interactive 3d plot
from mpl_toolkits.mplot3d import Axes3D # use this module for interactive 3d plot

sym = 'M20'

fig = plt.figure()
ax = fig.gca(projection='3d')# GCA = get current axes : this line is for static plot
#ax = Axes3D(fig) # use this ax for interactive 3d plot
ax.plot_trisurf(ready_df.loc[ready_df['Series']==sym, 'atr'],ready_df.loc[ready_df['Series']==sym, 'dte'],ready_df.loc[ready_df['Series']==sym, 'atm_tv'],
                        cmap=plt.cm.jet,edgecolor='black',linewidth=0.5)
ax.set_title('{} ATM Time Value by 1D ATR14 and Day to expiration'.format(sym))
ax.set_xlabel('Daily ATR 14')
ax.set_ylabel('DTE')
ax.set_zlabel('TV at ATM')
plt. tight_layout()

plt.show()

In [ ]:

### Current Working Cell

## Multi Series Static 3d Subplots and followed by a interactive 3d plot.


### Self Note 
- Linear Regression To ATM OTM pricing using ATR / Day to expire /etc. as features


In [ ]:
# Export the processed CSV
# ready_df.to_csv('ready_df.csv',index=False)